# Import Data:

In [1]:
#list the data - the csv files in this directory are a result of the API code below
!ls Data

2000_housevote_meta.csv   2010_senatevote_pos.csv
2000_housevote_pos.csv	  2011_housevote_meta.csv
2000_senatevote_meta.csv  2011_housevote_pos.csv
2000_senatevote_pos.csv   2011_senatevote_meta.csv
2001_housevote_meta.csv   2011_senatevote_pos.csv
2001_housevote_pos.csv	  2012_housevote_meta.csv
2001_senatevote_meta.csv  2012_housevote_pos.csv
2001_senatevote_pos.csv   2012_senatevote_meta.csv
2002_housevote_meta.csv   2012_senatevote_pos.csv
2002_housevote_pos.csv	  2013_housevote_meta.csv
2002_senatevote_meta.csv  2013_housevote_pos.csv
2002_senatevote_pos.csv   2013_senatevote_meta.csv
2003_housevote_meta.csv   2013_senatevote_pos.csv
2003_housevote_pos.csv	  2014_housevote_meta.csv
2003_senatevote_meta.csv  2014_housevote_pos.csv
2003_senatevote_pos.csv   2014_senatevote_meta.csv
2004_housevote_meta.csv   2014_senatevote_pos.csv
2004_housevote_pos.csv	  2015_housevote_meta.csv
2004_senatevote_meta.csv  2015_housevote_pos.csv
2004_senatevote_pos.csv   2015_senatevote_meta.csv
2005_

In [2]:
!ls

Capstone Project Proposal - 2.pdf  Data Wrangling.ipynb    README.md
Data				   PROPUBLICA_API_KEY.TXT


### Get Pro Publica API working

In [27]:
import requests

#Load ProPublica API key without exposing it to github. Request one yourself. They're free.
#https://projects.propublica.org/api-docs/congress-api/
APIfile = open('./PROPUBLICA_API_KEY.TXT',)
APIkey = APIfile.read().replace('\n', '')

#simple unit tests - will throw an error if API key is malformed
assert type(APIkey) == str
assert len(APIkey) == 40

#create API key header
headers = {'X-API-Key': APIkey}

#API request function
def API_Request(end_point, verbose=True):
    
    #send a get request to restful API, print status and return response as a dictionary
    response = requests.get(url+end_point, headers=headers)
    
    #raise error if status code isn't 200. 
    response.raise_for_status()
    
    #return json response interpreted as a dictionary. 
    return response.json()

#load base url and test endpoint
url = 'https://api.propublica.org/congress/v1'
test_endpoint = '/members/new.json'

#test that the API is working. 
new_members = API_Request(test_endpoint)
assert new_members['status'] == 'OK'
assert new_members['results'][0]['num_results'] == '20'
print(new_members['status'])

OK


### OK, The API is working.  Let's import voting data

In [43]:
import pandas as pd
import numpy as np
from retrying import retry
from datetime import datetime

@retry(stop_max_attempt_number=5) #we're going to try this function 5 times until it returns a value
def API_attempts(endpoint):
    '''takes and endpoint and requests the API with it for a given number of times'''  
    #if we can't extract the status we'll keep trying, and if it's OK return the repsonse
    
    #initialize status as an error, this will change if we can extract it.
    status = 'Error fetching API response'
    
    #try to get the API response and extract status without throwing an error on failure
    try:
        #send endpoint to API function for request
        response = API_Request(endpoint)
        
        #extract the status of the API request
        status = response['status']   
    except:
        pass
    
    #if the status is not OK:
    if status != 'OK':
        #Say what the status is
        print(status, end=' ')
        
        #state that we're retrying
        print('retrying')
        
        raise
    else: 
        #return the response if status is ok
        return response
    
        
#let's create a function that will import vote positions for a given rollcall vote.
def import_vote(congress, chamber, session, roll_call, verbose=False):
    '''imports vote details and member positions of a given roll_call vote
        takes congress number 102-116 for House, 80-116 for Senate
        chamber(house|senate) session 1 for odd 2 for even-numbered years
        returns a dicitonary of vote positions and API status'''
    #construct endpoint for API request
    for x in [congress, chamber, session, roll_call]:
        x = str(x)#                 
    call_endpoint = '/{}/{}/sessions/{}/votes/{}.json'.format(congress, chamber.lower(), session, roll_call)
    
    #let's attempt a few times to account for internet burps
    call_response = API_attempts(call_endpoint)  
        
    #try to extract and return the vote positions
    try:
        vote = call_response['results']['votes']['vote']
        return vote
    except:
        return {}
    

def positions_df(metalist):
    '''return a dataframe of postions for a given list of vote metadata'''
    
    #initialize dataframe
    dfp = pd.DataFrame()
    
    #for every position in the list
    for congress, chamber, session, roll_call in metalist:
        
        #request the roll_call vote from the API
        vote = import_vote(congress, chamber, session, roll_call)
        
        #extract the positions from the rollcall vote data
        positions = vote.pop('positions')
        
        #turn the postions into a dataframe
        call_positions = pd.DataFrame(positions)
        
        #if both dataframes we are going to merge are not empty
        if not dfp.empty and not call_positions.empty:
            
            #handle duplicated positions for malformed returns
            dups = call_positions.duplicated()
            dups = dups.sum()
            if dups > 0: 
                call_postions = call_postions.drop_duplicates()
                print('dropped{}duplicates'.format(dups))
        
            #MERGE THE POSITIONS FOR A GIVEN ROLL CALL VOTE INTO DATAFRAME <- core of function 
            dfp = pd.merge(dfp, call_positions, on=mergelst, how='outer')
            
            #let us know it's working
            print('.', end='')
            
        #If dfp is empty this is the first run and we need to initialize it
        elif dfp.empty: 
            #make it equal to call_positions
            dfp = call_positions
            
            #make a list of the columns that will be used to merge the call positions
            mergelst = dfp.columns.tolist()[:-1]
            
            #move party and state to front of merglist
            for x in ['state','party']:
                mergelst.insert(0, mergelst.pop(mergelst.index(x)))
            
            #alert that we've started
            print('\ndf created on', congress, chamber, session, roll_call, end=' ')
        
        #handle empty postitions dataframe:
        elif call_positions.empty:
            
            #let us know there was a problem
            print('\nno vote postions available for', congress, chamber, session, roll_call, end=' ')
            
            #append a series of Nan values
            ht = dfp.shape[0]
            nans = pd.Series([np.nan for _ in range(ht)])
            print ('dfp.shape:', dfp.shape)
            print ('Appending {}'.format(len(nans)))
            dfp = pd.concat([dfp, nans], axis = 1)
            print(dfp.iloc[:3,-4:])
        
        #handle unknown issue
        else:
            print('\nunknown error for', congress, chamber, session, roll_call, end=' ')
    
    #create a multindex of partymembers
    dfp = dfp.set_index(mergelst).transpose()
    
    #after transposing use the metalist as the index
    dfp.index = metalist
    
    #return the complete data frame
    return dfp 

def import_month(year, chamber, month):
    '''import vote metadata for a given year or month and return a dataframe '''
    
    #Stop everthing if we are using a year before 1989 or after now
    assert 1989 <= year <= int(datetime.now().year), "year out of range"
    
    #construct endpoint
    call_endpoint = '/{}/votes/{}/{}.json'.format(chamber.lower(), str(year), str(month))
    
    #use endpoint to request the API robustly with attempts function
    response = API_attempts(call_endpoint)
    
    #Extract the votes from the response
    votes = response['results']['votes']

    #create a dataframe out of the votes
    votesdf = pd.DataFrame(votes)
    
    if not votesdf.empty:
        votesdf = votesdf.set_index(['congress','chamber','session','roll_call']).sort_index()
    
    #return the months dataframe
    return votesdf

def import_year(year, chamber):
    '''import votes based on dates, return a dataframe of appended votes'''
    
    
    #So we know we started:
    print('importing', chamber, year, end=' ')
    #loop through the months 
    for month in range(1,13):
        
        #import the first month
        if month == 1:
            yeardf = import_month(year, chamber, month)
       
        #import the month of votes as a df and append it to the list 
        else:
            df = import_month(year, chamber, month)
            if not df.empty:
                yeardf = yeardf.append(df)
            else:
                print('skip', end=' ')
        #print something so we know it's working
        print(month, end=' ')
        
    #return the dataframe
    return yeardf

def save_years(start_y, end_y):
    '''take a range of years and save a series of dataframes for those years in both the hosue and senate'''
    #iterate through the range of years
    for year in range(start_y, end_y+1):
        #alternate between senate and house
        for chamber in ['senate','house']:
            #import vote metadata dataframe
            votemetadf = import_year(year, chamber)
            
            #import vote positions for given dataframe
            voteposdf = positions_df(votemetadf.index)
            
            #construct paths
            #pathhead = 'Data/'+str(year)+'_'+chamber+'vote_'
            pathhead = 'Data/{}_{}vote_'.format(str(year), chamber)
            metapath = pathhead + 'meta.csv'
            pospath = pathhead + 'pos.csv'
            
            #announce what we're doing
            #print('\nsaving '+str(year)+' '+chamber+' as '+metapath+' and '+pospath)
            print('\nsaving {} {} as {} and {}'.format(str(year), chamber, metapath, pospath))
        
            #save data frames as csv in data/csv directory
            votemetadf.to_csv(metapath)
            voteposdf.to_csv(pospath)
            
            #save data frames as pickle in data/pickle directory?
            
            
#tests - comment out when working

#senate_month_test = import_month(2019, 'senate', 1)
#senate_month_test

#senate_2019_test = import_year(2019, 'senate')
#senate_2019_test

#positions = positions_df(senate_2019_test.index)
#positions

#save_years(2019, 2019)

In [35]:
save_years(2016,2017)

importing senate 2016 1 2 3 4 5 6 7 skip 8 9 skip 10 11 12 
df created on 114 Senate 2 1 ..................................................................................................................................................................
saving 2016 senate as Data/2016_senatevote_meta.csv and Data/2016_senatevote_pos.csv
importing house 2016 1 2 3 4 5 6 7 skip 8 9 skip 10 11 12 
df created on 114 House 2 1 ................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [43]:
save_years(2009,2009)

importing senate 2009 1 2 3 4 5 6 7 8 9 10 11 12 
df created on 111 Senate 1 1 ............................................................................................................................................................................................................................................................................................................................................................................................................
saving 2009 senate as Data/2009_senatevote_meta.csv and Data/2009_senatevote_pos.csv
importing house 2009 1 2 3 4 5 6 7 skip 8 9 10 11 12 
df created on 111 House 1 1 .....................................................................................................................................................................................................................................................................................................................................................................

In [39]:
test_dfsenate = pd.read_csv('Data/2019_senatevote_pos.csv', header = [0,1,2,3,4], index_col = [0,1,2,3])
test_dfsenate

party                                            R             D  \
state                                           TN            WI   
dw_nominate                                  0.323        -0.521   
member_id                                  A000360       B001230   
name                               Lamar Alexander Tammy Baldwin   
congress chamber session roll_call                                 
116      Senate  1       1                     Yes            No   
                         2                     Yes            No   
                         3                     Yes            No   
                         4                     Yes            No   
                         5                      No           Yes   
                         6                      No           Yes   
                         7              Not Voting            No   
                         8                      No           Yes   
                         9                     Yes            No   
                         10                    Yes           Yes   
                         11                    Yes            No   
                         12                    Yes            No   
                         13             Not Voting            No   
                         14                    Yes            No   
                         15                    Yes            No   
                         16                    Yes            No   
                         17                    Yes           Yes   
                         18                    Yes           Yes   
                         19                    Yes           Yes   
                         20                    Yes           Yes   
                         21                    Yes           Yes   
                         22                    Yes           Yes   
                         23                    Yes            No   
                         24                    Yes            No   
                         25                    Yes           Yes   
                         26                    Yes           Yes   
                         27                    Yes            No   
                         28                    Yes            No   
                         29                    Yes            No   
                         30                    Yes           Yes   
...                                            ...           ...   
                         98                    Yes           Yes   
                         99                    Yes            No   
                         100                   Yes           Yes   
                         101                   Yes           Yes   
                         102                   Yes           Yes   
                         103                   Yes            No   
                         104                   Yes            No   
                         105                   Yes            No   
                         106                   Yes            No   
                         107                   Yes            No   
                         108                   Yes            No   
                         109                   Yes            No   
                         110                   Yes            No   
                         111                   Yes            No   
                         112                   Yes           Yes   
                         113                   Yes            No   
                         114                   Yes            No   
                         115                   Yes           Yes   
                         116            Not Voting            No   
                         117                   Yes    Not Voting   
                         118                   Yes            No   
                         119                   Yes      

In [38]:
test_dfhouse = pd.read_csv('Data/2019_housevote_pos.csv', header = [0,1,2,3,4,5], index_col = [0,1,2,3])
test_dfhouse

party                                          R          D  \
state                                         LA         NC   
district                                       5         12   
dw_nominate                                0.514     -0.469   
member_id                                A000374    A000370   
name                               Ralph Abraham Alma Adams   
congress chamber session roll_call                            
116      House   1       1               Present    Present   
                         2              McCarthy     Pelosi   
                         3                    No        Yes   
                         4                    No        Yes   
                         5                   Yes         No   
                         6                    No        Yes   
                         7                    No        Yes   
                         8                   Yes         No   
                         9                    No        Yes   
                         10                  Yes         No   
                         11                   No        Yes   
                         12                  Yes        Yes   
                         13                  Yes        Yes   
                         14                  Yes        Yes   
                         15                   No        Yes   
                         16                   No        Yes   
                         17                   No        Yes   
                         18                  Yes        Yes   
                         19                   No        Yes   
                         20                  Yes         No   
                         21                   No        Yes   
                         22                  Yes         No   
                         23                   No        Yes   
                         24                  Yes         No   
                         25                   No        Yes   
                         26           Not Voting         No   
                         27           Not Voting        Yes   
                         28           Not Voting        Yes   
                         29           Not Voting        Yes   
                         30                  Yes        Yes   
...                                          ...        ...   
                         188                 Yes        Yes   
                         189                  No        Yes   
                         190                  No        Yes   
                         191          Not Voting        Yes   
                         192          Not Voting         No   
                         193          Not Voting        Yes   
                         194          Not Voting        Yes   
                         195          Not Voting         No   
                         196          Not Voting        Yes   
                         197          Not Voting        Yes   
                         198          Not Voting        Yes   
                         199          Not Voting        Yes   
                         200          Not Voting        Yes   
                         201          Not Voting         No   
                         202          Not Voting        Yes   
                         203          Not Voting        Yes   
                         204          Not Voting        Yes   
                         205          Not Voting        Yes   
                         206          Not Voting        Yes   
                         207          Not Voting        Yes   
                         208          Not Voting        Yes   
                         209          Not Voting        Yes   
                         210          Not Voting         No   
                         211          Not Voting        Yes   
                         212          Not Voting        Yes   
                         213   

In [41]:
testmetahouse = pd.read_csv('Data/2019_housevote_meta.csv', index_col = [0,1,2,3])
testmetahouse

bill  \
congress chamber session roll_call                                                      
116      House   1       1          {'bill_id': 'quorum-116', 'number': 'QUORUM', ...   
                         2                                                         {}   
                         3          {'bill_id': 'hres5-116', 'number': 'H.RES.5', ...   
                         4          {'bill_id': 'hres5-116', 'number': 'H.RES.5', ...   
                         5          {'bill_id': 'hres5-116', 'number': 'H.RES.5', ...   
                         6          {'bill_id': 'hres5-116', 'number': 'H.RES.5', ...   
                         7          {'bill_id': 'hres6-116', 'number': 'H.RES.6', ...   
                         8          {'bill_id': 'hjres1-116', 'number': 'H.J.RES.1...   
                         9          {'bill_id': 'hjres1-116', 'number': 'H.J.RES.1...   
                         10         {'bill_id': 'hr21-116', 'number': 'H.R.21', 's...   
                         11         {'bill_id': 'hr21-116', 'number': 'H.R.21', 's...   
                         12         {'bill_id': 'hres6-116', 'number': 'H.RES.6', ...   
                         13         {'bill_id': 'hr269-116', 'number': 'H.R.269', ...   
                         14         {'bill_id': 'hr251-116', 'number': 'H.R.251', ...   
                         15         {'bill_id': 'journal-116', 'number': 'JOURNAL'...   
                         16         {'bill_id': 'hres28-116', 'number': 'H.RES.28'...   
                         17         {'bill_id': 'hres28-116', 'number': 'H.RES.28'...   
                         18         {'bill_id': 'hr226-116', 'number': 'H.R.226', ...   
                         19         {'bill_id': 'hres6-116', 'number': 'H.RES.6', ...   
                         20         {'bill_id': 'hr264-116', 'number': 'H.R.264', ...   
                         21         {'bill_id': 'hr264-116', 'number': 'H.R.264', ...   
                         22         {'bill_id': 'hr267-116', 'number': 'H.R.267', ...   
                         23         {'bill_id': 'hr267-116', 'number': 'H.R.267', ...   
                         24         {'bill_id': 'hr265-116', 'number': 'H.R.265', ...   
                         25         {'bill_id': 'hr265-116', 'number': 'H.R.265', ...   
                         26         {'bill_id': 'hr266-116', 'number': 'H.R.266', ...   
                         27         {'bill_id': 'hr266-116', 'number': 'H.R.266', ...   
                         28         {'bill_id': 's24-116', 'number': 'S.24', 'spon...   
                         29         {'bill_id': 'hr221-116', 'number': 'H.R.221', ...   
                         30         {'bill_id': 'hr116-116', 'number': 'H.R.116', ...   
...                                                                               ...   
                         188        {'bill_id': 'hr1520-116', 'number': 'H.R.1520'...   
                         189        {'bill_id': 'hres357-116', 'number': 'H.RES.35...   
                         190        {'bill_id': 'hres357-116', 'number': 'H.RES.35...   
                         191        {'bill_id': 'hr986-116', 'number': 'H.R.986', ...   
                         192        {'bill_id': 'hr986-116', 'number': 'H.R.986', ...   
                         193        {'bill_id': 'hr986-116', 'number': 'H.R.986', ...   
                         194        {'bill_id': 'hr986-116', 'number': 'H.R.986', ...   
                         195        {'bill_id': 'hr986-116', 'number': 'H.R.986', ...   
                         196        {'bill_id': 'hr986-116', 'number': 'H.R.986', ...   
                         197        {'bill_id': 'hr2157-116', 'number': 'H.R.2157'...   
                         198        {'bill_id': 'hr2157-116', 'number': 'H.R.2157'...   
                         199        {'bill_id': 'hr2157-116', 'number': 'H.R.2157'...   
                         200        {'bill_id': 'hr2157-116', 'number': 'H.R

In [42]:
testmetasenate = pd.read_csv('Data/2019_senatevote_meta.csv', index_col = [0,1,2,3])
testmetasenate

bill  \
congress chamber session roll_call                                                      
116      Senate  1       1          {'bill_id': 's1-116', 'number': 'S.1', 'sponso...   
                         2          {'bill_id': 's1-116', 'number': 'S.1', 'sponso...   
                         3          {'bill_id': 's1-116', 'number': 'S.1', 'sponso...   
                         4          {'bill_id': 'sjres2-116', 'number': 'S.J.RES.2...   
                         5          {'bill_id': 'sjres2-116', 'number': 'S.J.RES.2...   
                         6          {'bill_id': 'sjres2-116', 'number': 'S.J.RES.2...   
                         7          {'bill_id': 's109-116', 'number': 'S.109', 'sp...   
                         8          {'bill_id': 'hr268-116', 'number': 'H.R.268', ...   
                         9          {'bill_id': 'hr268-116', 'number': 'H.R.268', ...   
                         10         {'bill_id': 'hr268-116', 'number': 'H.R.268', ...   
                         11         {'bill_id': 's1-116', 'number': 'S.1', 'sponso...   
                         12         {'bill_id': 's1-116', 'number': 'S.1', 'sponso...   
                         13         {'bill_id': 's1-116', 'number': 'S.1', 'sponso...   
                         14         {'bill_id': 's1-116', 'number': 'S.1', 'sponso...   
                         15         {'bill_id': 's1-116', 'number': 'S.1', 'sponso...   
                         16         {'bill_id': 's1-116', 'number': 'S.1', 'sponso...   
                         17         {'bill_id': 's47-116', 'number': 'S.47', 'spon...   
                         18         {'bill_id': 's47-116', 'number': 'S.47', 'spon...   
                         19         {'bill_id': 's47-116', 'number': 'S.47', 'spon...   
                         20         {'bill_id': 's47-116', 'number': 'S.47', 'spon...   
                         21         {'bill_id': 's47-116', 'number': 'S.47', 'spon...   
                         22         {'bill_id': 's47-116', 'number': 'S.47', 'spon...   
                         23                                                        {}   
                         24                                                        {}   
                         25         {'bill_id': 'hjres31-116', 'number': 'H.J.RES....   
                         26         {'bill_id': 'hjres31-116', 'number': 'H.J.RES....   
                         27         {'bill_id': 's311-116', 'number': 'S.311', 'sp...   
                         28                                                        {}   
                         29                                                        {}   
                         30                                                        {}   
...                                                                               ...   
                         98                                                        {}   
                         99                                                        {}   
                         100                                                       {}   
                         101                                                       {}   
                         102                                                       {}   
                         103                                                       {}   
                         104                                                       {}   
                         105                                                       {}   
                         106                                                       {}   
                         107                                                       {}   
                         108                                                       {}   
                         109                                                       {}   
                         110                                                